In [1]:
class YouCook2VideoLoader:
    """YouCook2 video dataset loader for VJEPA2 world model experiments."""
    def __init__(self, data_path, frames_per_clip=16, img_size=384):
        self.data_path = Path(data_path)
        self.frames_per_clip = frames_per_clip
        self.img_size = img_size
        # Find all video files
        self.video_files = list(self.data_path.rglob("*.mp4"))
        print(f"Found {len(self.video_files)} video files")
        if len(self.video_files) == 0:
            raise ValueError(f"No video files found in {data_path}")
    def load_video_frames(self, video_path, start_frame=None):
        """Load frames from a video file."""
        try:
            vr = VideoReader(str(video_path), num_threads=-1, ctx=cpu(0))
            vlen = len(vr)
            if vlen < self.frames_per_clip:
                print(f":warning: Video too short: {vlen} < {self.frames_per_clip}")
                return None
            # Sample frames
            if start_frame is None:
                start_frame = np.random.randint(0, vlen - self.frames_per_clip + 1)
            end_frame = start_frame + self.frames_per_clip
            indices = np.arange(start_frame, end_frame)
            # Load frames
            vr.seek(0)
            frames = vr.get_batch(indices).asnumpy()  # Shape: (T, H, W, C)
            # Resize frames
            resized_frames = []
            for frame in frames:
                resized = cv2.resize(frame, (self.img_size, self.img_size))
                resized_frames.append(resized)
            frames = np.stack(resized_frames)  # (T, H, W, C)
            return frames, indices
        except Exception as e:
            print(f":x: Error loading video {video_path}: {e}")
            return None
    def get_random_video_clip(self):
        """Get a random video clip from the dataset."""
        max_attempts = 10
        for _ in range(max_attempts):
            video_path = np.random.choice(self.video_files)
            result = self.load_video_frames(video_path)
            if result is not None:
                frames, indices = result
                return frames, video_path, indices
        raise RuntimeError("Could not load any valid video after multiple attempts")
# Initialize YouCook2 loader
dataset_path = "/workspace/scene_captioner/data/"
video_loader = YouCook2VideoLoader(dataset_path, frames_per_clip=16, img_size=384)
print(f":video_camera: YouCook2 loader initialized with {len(video_loader.video_files)} videos")
print(f":clapper: Frame size: {video_loader.img_size}x{video_loader.img_size}")
print(f":1234: Frames per clip: {video_loader.frames_per_clip}")

NameError: name 'Path' is not defined